In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import layers, models, losses, optimizers
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import warnings
# print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_physical_devices('GPU'))

2023-04-21 00:08:05.023226: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-21 00:08:06.639772: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-21 00:08:06.640474: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-21 00:08:07.038700: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1747] Found device 0 with properties: 
pciBusID: 0000:87:00.0 name: NVIDIA A100-SXM4-80GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 79.18GiB deviceMemoryBandwidth: 1.85TiB/s
2023-04-21 00:08:07.038742: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-04-21 00:08:07.041952: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-04-21 00:08:07.041989: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2

In [2]:
# import dataset
datapath = "../wildfire_dataset.nc"
wildfire_dataset = xr.open_dataset(datapath, engine="netcdf4")
# print(wildfire_dataset)
feature_list = wildfire_dataset.data_vars
feature_nums = len(feature_list)

# maybe predict more than one thing later, but for not only try to predict burned areas
remove_label = ["burned_areas", "ignition_points", "number_of_fires"]
X_label = [label for label in feature_list if label not in remove_label]
y_label = "burned_areas"


/usr/local/lib/python3.8/dist-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
# take the first 5 time steps for all x and y to try creating a smaller dataset
num_samples = 10
timesteps_per_sample = 5
timesteps = num_samples*timesteps_per_sample
wf_dataset_head = wildfire_dataset.head(indexers={"time": timesteps})

############# Normalize Data Here ####################

wf_dataset_X = wf_dataset_head[X_label]
wf_ds_norm_X = wf_dataset_X # REPLACE LATER WITH NORMALIZED/CORRECTLY STRUCTURED DATA

wf_dataset_y = wf_dataset_head[y_label] 
wf_ds_norm_y = wf_dataset_y # REPLACE LATER WITH NORMALIZED/CORRECTLY STRUCTURED DATA

######################################################

# Create the y into a numpy matrix of shape (time, x, y)
wf_dataset_y_np = wf_dataset_y.to_numpy()
wf_dataset_y_np = np.transpose(wf_dataset_y_np, (0,2,1))

# Create the X into a numpy matrix of shape (time, x, y)
wf_dataset_X_np = wf_dataset_X[list(wf_dataset_X.data_vars)[0]].to_numpy()
wf_dataset_X_np = np.transpose(wf_dataset_X_np, (0,2,1))
wf_dataset_X_np = np.expand_dims(wf_dataset_X_np, 3)

In [4]:
# Takes each feature of the xarray Dataset and converts it into a DataArray 
# Also appends it into the new np array to make shape of (time x, y, features)
for index, feature in enumerate(list(wf_dataset_X.data_vars)):
    if(index!=0):
        # Since wf_dataset_X_np is already initiaklized with the first element, skip
        new_np_arr = wf_dataset_X[feature].to_numpy()
        if(len(new_np_arr.shape) == 2):
            # If a feature doesn't contain a time dimension (n), we extend the 2d matrix to 3d with copy of matrix n times
            # Might be able to use numpy broadcast instead
            new_np_arr = np.repeat(new_np_arr[:, :, np.newaxis], timesteps, axis=2)
            # Transpose feature to "time", "x", "y" format
            new_np_arr = np.transpose(new_np_arr)
        else:
            # Transpose feature to "time", "x", "y" format
            new_np_arr = np.transpose(new_np_arr, (0,2,1))
        if (np.isnan(new_np_arr).all()):
            # Precaution to alert if a feature has all NaN values
            warnings.warn(str(feature) + " feature's values are all NaNs")
        wf_dataset_X_np = np.concatenate((wf_dataset_X_np, np.expand_dims(new_np_arr, axis=3)), axis=3)
    print(feature)
    print(wf_dataset_X_np.shape)

ndvi
(50, 1253, 983, 1)
evi
(50, 1253, 983, 2)
et
(50, 1253, 983, 3)
lst_day
(50, 1253, 983, 4)
lst_night
(50, 1253, 983, 5)
fapar
(50, 1253, 983, 6)
lai
(50, 1253, 983, 7)
max_u10
(50, 1253, 983, 8)
max_v10
(50, 1253, 983, 9)
max_d2m
(50, 1253, 983, 10)
max_t2m
(50, 1253, 983, 11)
max_sp
(50, 1253, 983, 12)
max_tp
(50, 1253, 983, 13)
min_u10
(50, 1253, 983, 14)
min_v10
(50, 1253, 983, 15)
min_d2m
(50, 1253, 983, 16)
min_t2m
(50, 1253, 983, 17)
min_sp
(50, 1253, 983, 18)
min_tp
(50, 1253, 983, 19)
avg_u10
(50, 1253, 983, 20)
avg_v10
(50, 1253, 983, 21)
avg_d2m
(50, 1253, 983, 22)
avg_t2m
(50, 1253, 983, 23)
avg_sp
(50, 1253, 983, 24)
avg_tp
(50, 1253, 983, 25)
smian
(50, 1253, 983, 26)
sminx
(50, 1253, 983, 27)
fwi
(50, 1253, 983, 28)
max_wind_u10
(50, 1253, 983, 29)
max_wind_v10
(50, 1253, 983, 30)
max_wind_speed
(50, 1253, 983, 31)
max_wind_direction
(50, 1253, 983, 32)
max_rh
(50, 1253, 983, 33)
min_rh
(50, 1253, 983, 34)
avg_rh
(50, 1253, 983, 35)
CLC_2006
(50, 1253, 983, 36)
CLC_2

In [5]:
# rearrange so that it matches what keras expects (time, features, x, y) instead of (time, x, y, features)
wf_dataset_X_np = np.moveaxis(wf_dataset_X_np, 3, 1)
print("wf_dataset_X_np.shape", wf_dataset_X_np.shape)
print("wf_dataset_y_np.shape", wf_dataset_y_np.shape)

wf_dataset_X_np.shape (50, 87, 1253, 983)
wf_dataset_y_np.shape (50, 1253, 983)


In [6]:
if 1 in wf_dataset_y_np:
    print("Fire exists") 
print("Output classes: ", np.unique(wf_dataset_y_np))
# class_weights = class_weight.compute_class_weight(class_weight = "balanced", classes = np.unique(wf_dataset_y_np), y = wf_dataset_y_np)
# print(class_weights)

Output classes:  [0.]


In [7]:
# Create samples (samples, time, features, x, y)
# Each samples are 4 days with 2 day overlap between each one
wf_dataset_X_np = np.expand_dims(wf_dataset_X_np, axis=0)
wf_dataset_X_np = np.reshape(wf_dataset_X_np, (num_samples, timesteps_per_sample, wf_dataset_X_np.shape[2], wf_dataset_X_np.shape[3], wf_dataset_X_np.shape[4]))

print(wf_dataset_X_np.shape)

(10, 5, 87, 1253, 983)


In [8]:
# train test split (70/30 split)
# split along axis 0
wf_dataset_X_np_split = np.split(wf_dataset_X_np, [7, 10])
wf_dataset_y_np_split = np.split(wf_dataset_y_np, [7, 10])
X_train = wf_dataset_X_np_split[0]
X_test = wf_dataset_X_np_split[1]
y_train = wf_dataset_y_np_split[0]
y_test = wf_dataset_y_np_split[1]

In [9]:
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_train: ", y_train.dtype)
print("y_test: ", y_test.shape)
print("input shape: ", X_train.shape[-4:])

X_train:  (7, 5, 87, 1253, 983)
X_test:  (3, 5, 87, 1253, 983)
y_train:  (7, 1253, 983)
y_train:  float32
y_test:  (3, 1253, 983)
input shape:  (5, 87, 1253, 983)


In [10]:
def build_ConvLSTM():
    convlstm = models.Sequential()
    convlstm.add(layers.Input(shape=X_train.shape[-4:]))
    convlstm.add(layers.ConvLSTM2D(filters=256, kernel_size=(5,5), return_sequences=True))
    convlstm.add(layers.BatchNormalization())
    convlstm.add(layers.ConvLSTM2D(filters=128, kernel_size=(3,3), return_sequences=True))
    convlstm.add(layers.BatchNormalization())
    convlstm.add(layers.ConvLSTM2D(filters=64, kernel_size=(2,2), return_sequences=True))
    convlstm.add(layers.BatchNormalization())
    convlstm.add(layers.ConvLSTM2D(filters=32, kernel_size=(1,1), return_sequences=True))
    convlstm.add(layers.Conv3D(filters=1, kernel_size=(5, 80, 1246), activation="sigmoid"))
    convlstm.compile(
        loss=losses.binary_crossentropy, optimizer=optimizers.Adam(),
        # metrics=[tf.keras.metrics.Accuracy()]
    )
    return convlstm

In [11]:
model = build_ConvLSTM()
print(model.summary())
epochs = 10
batch_size = 5
# y_train = np.expand_dims(y_train, axis=0)

2023-04-21 00:12:50.530264: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-21 00:12:50.535783: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1747] Found device 0 with properties: 
pciBusID: 0000:87:00.0 name: NVIDIA A100-SXM4-80GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 79.18GiB deviceMemoryBandwidth: 1.85TiB/s
2023-04-21 00:12:50.535818: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-04-21 00:12:50.535879: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-04-21 00:12:50.535894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2023-04-21 00:12:50.535907: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 5, 83, 1249, 256)  31719424  
_________________________________________________________________
batch_normalization (BatchNo (None, 5, 83, 1249, 256)  1024      
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 5, 81, 1247, 128)  1769984   
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 81, 1247, 128)  512       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 5, 80, 1246, 64)   196864    
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 80, 1246, 64)   256       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 5, 80, 1246, 32)   1

In [12]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=True)

2023-04-21 00:13:18.567616: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-21 00:13:18.586443: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2245870000 Hz


Epoch 1/10


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
        outputs = model.t